# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path

In [3]:
print(tf.__version__)

2.5.0


In [4]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
mlflow.set_tracking_uri("file:///E:/Github/thesis/Dissertation/mlruns")

In [6]:
if tf.test.gpu_device_name(): 
    
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [7]:
#makes all outputs be in float format rather than exponentials
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [8]:
#Set Parameters
IMG_SIZE = 224 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
log_dir=f'logs\\{MODEL_NAME}'
TENSORBOARD = TensorBoard(log_dir)

In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [10]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [11]:
# ###Directory for MacBook
# mac_URL = '/Volumes/T7/Uni/Github/thesis/flowerDataset/Flower'
# mac_labels = '/Volumes/T7/Uni/Github/thesis/flowerDataset/imagelabels.mat'
# mac_dataSplit = '/Volumes/T7/Uni/Github/thesis/flowerDataset/setid.mat'
# mac_loaded_images = '/Volumes/T7/Uni/loaded_images.npy'
# mac_image_train='/Volumes/T7/Uni/image_train.npy'
# mac_label_train = '/Volumes/T7/Uni/label_train.npy'
# mac_image_test = '/Volumes/T7/Uni/image_test.npy'
# mac_label_test = '/Volumes/T7/Uni/label_test.npy'
# mac_image_val = '/Volumes/T7/Uni/image_val.npy'
# mac_label_val = '/Volumes/T7/Uni/label_val.npy'

# ###Directory for PC
# pc_URL = 'E:/Github/thesis/flowerDataset/Flower'
# pc_labels = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
# pc_dataSplit = 'E:/Github/thesis/flowerDataset/setid.mat'
# pc_loaded_images = 'E:/Dissertation/data/loaded_images.npy'
# pc_image_train = 'E:/Dissertation/data/image_train.npy'
# pc_label_train = 'E:/Dissertation/data/label_train.npy'
# pc_image_test = 'E:/Dissertation/data/image_test.npy'
# pc_label_test = 'E:/Dissertation/data/label_test.npy'
# pc_image_val = 'E:/Dissertation/data/image_val.npy'
# pc_label_val = 'E:/Dissertation/data/label_val.npy'

In [12]:
def get_directories():
    if os.path.exists(mac_URL) and os.path.exists(mac_labels) and os.path.exists(
            mac_dataSplit) and os.path.exists(mac_loaded_images):
        return mac_URL, mac_labels, mac_dataSplit, mac_loaded_images, mac_image_train, mac_label_train, mac_image_test, mac_label_test,mac_image_val, mac_label_val 
    else:
        return pc_URL, pc_labels, pc_dataSplit, pc_loaded_images, pc_image_train, pc_label_train, pc_image_test, pc_label_test, pc_image_val, pc_label_val

In [13]:
#URL_dir, labels_dir, dataSplit_dir, loaded_images_dir, image_train_dir, label_train_dir, image_test_dir, label_test_dir, image_val_dir, label_val_dir = get_directories()

In [14]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
dataSplit_dir = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/data/loaded_imagesbw.npy'
image_train_dir ='E:/Dissertation/data/image_trainbw.npy'
label_train_dir = 'E:/Dissertation/data/label_train.npy'
image_test_dir = 'E:/Dissertation/data/image_testbw.npy'
label_test_dir = 'E:/Dissertation/data/label_test.npy'
image_val_dir = 'E:/Dissertation/data/image_valbw.npy'
label_val_dir = 'E:/Dissertation/data/label_val.npy'

In [15]:
print(labels_dir)

E:/Github/thesis/flowerDataset/imagelabels.mat


### Load DataSet and Labels

In [16]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [17]:
df = pd.DataFrame(data_labels)

In [18]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [19]:
labels

array([77, 77, 77, ..., 62, 62, 62], dtype=uint8)

In [20]:
len(labels)

8189

In [21]:
UNIQUE_LABELS = np.unique(labels)
UNIQUE_LABELS

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102], dtype=uint8)

In [22]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,1])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [62]:
#No need to run this cell if image already loaded.
load_images_from_folder(URL_dir)

[array([[[118],
         [110],
         [ 97],
         ...,
         [ 17],
         [ 22],
         [ 27]],
 
        [[124],
         [115],
         [100],
         ...,
         [ 13],
         [ 17],
         [ 21]],
 
        [[133],
         [121],
         [105],
         ...,
         [ 10],
         [ 14],
         [ 17]],
 
        ...,
 
        [[140],
         [132],
         [121],
         ...,
         [ 54],
         [ 56],
         [ 57]],
 
        [[146],
         [134],
         [126],
         ...,
         [ 51],
         [ 56],
         [ 55]],
 
        [[148],
         [136],
         [129],
         ...,
         [ 51],
         [ 56],
         [ 55]]], dtype=uint8),
 array([[[41],
         [45],
         [46],
         ...,
         [25],
         [26],
         [39]],
 
        [[48],
         [47],
         [52],
         ...,
         [25],
         [26],
         [39]],
 
        [[56],
         [54],
         [54],
         ...,
         [25],
      

In [63]:
#if images are already loaded, just load the file here rather than running function again.
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [64]:
#shows the shape of the image_data, the number of images, the dimensions and number of colour channels
image_data.shape

(8189, 224, 224, 1)

In [65]:
image_data[0].shape

(224, 224, 1)

In [66]:
#combine image with labels
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [67]:
#image_data, labels = randomised_dataset(image_data, labels)

In [68]:
print(image_data.shape)

(8189, 224, 224, 1)


In [69]:
# # Label Encoder

mlb = LabelBinarizer()
converted_labels = np.array(mlb.fit_transform(labels))

In [70]:
converted_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [71]:
def convert_label(labels):
    new_labels = []
    for i in range(len(labels)):
        new_labels.insert(i, [labels[i]])
    return np.array(new_labels)

In [72]:
#converted_labels = convert_label(labels)

In [73]:
#converted_labels.shape

In [74]:
len(converted_labels)

8189

In [75]:
converted_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Training and Testing Split

In [76]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [77]:
split_data(image_data,converted_labels)

image_train (6141, 224, 224, 1)
label_train (6141, 102)
image_test (820, 224, 224, 1)
label_test (820, 102)
image_val (1228, 224, 224, 1)
label_val (1228, 102)


In [78]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [79]:
image_test.shape

(820, 224, 224, 1)

In [80]:
label_val[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Feature Scaling

In [81]:
#converts the images to the range of 0 - 1.0 
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [82]:
image_train.shape

(6141, 224, 224, 1)

In [83]:
label_train.shape

(6141, 102)

In [84]:
label_test.shape

(820, 102)

In [85]:
len(image_test)

820

In [86]:
label_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [87]:
# number of classes/ flowers
NUM_CLASSES = 102
NUM_CLASSES

102

In [88]:
label_train.shape

(6141, 102)

In [89]:
label_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

# Custom CNN model

In [90]:
# name of model
name_model = 'test-{}'.format(int(time.time()))

In [91]:
batch_size = 32
epochs = 300
verbose= 2
act = 'relu'

In [92]:
def create_cnn_model(act):
    # tf.reset_default_graph()
    model = Sequential()
    input_shape = (IMG_SIZE, IMG_SIZE, 1)
    
    # INPUT LAYER
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation(act))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # HIDDEN LAYER 1
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # HIDDEN LAYER 2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 3
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 4
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # Fully Connected
    model.add(Flatten()) # converts the 3D feature maps to 1D feature vectors
    model.add(Dense(256))
    model.add(Activation(act))
    model.add(Dropout(0.25)) # reduces overfitting

    # OUTPUT LAYER
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [93]:
model = create_cnn_model(act)

In [94]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      320       
_________________________________________________________________
activation_7 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
activation_8 (Activation)    (None, 220, 220, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 110, 110, 32)      128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 64)     

In [95]:
#history = model.fit(image_train, label_train, batch_size=64, epochs=100, validation_data=(image_val, label_val), verbose=1, callbacks = [tensorboard])

In [96]:
#saving model
#model.save(f'{MODEL_PATH}')

In [97]:
#loading model
#model =  tf.keras.models.load_model(f'{MODEL_PATH}')
#model = tf.keras.models.load_model(f'E:/GoogleSync/Masters/Dissertation/models/24-Jun-2021') 

# Evaluation

In [98]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [99]:
EXPERIMENT_NAME = "custom-model-tests"
RUN_NAME = "default_cnn_BWgrayscale"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        
        
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        #mlflow.keras.log_model(model, "standardCNN")        

2021/08/07 18:17:12 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/300


192/192 - 16s - loss: 7.3618 - accuracy: 0.0314 - val_loss: 19.2358 - val_accuracy: 0.0163
Epoch 2/300
192/192 - 6s - loss: 4.7738 - accuracy: 0.0604 - val_loss: 7.4713 - val_accuracy: 0.0277
Epoch 3/300
192/192 - 6s - loss: 4.2851 - accuracy: 0.0944 - val_loss: 4.6915 - val_accuracy: 0.0603
Epoch 4/300
192/192 - 6s - loss: 4.0485 - accuracy: 0.1264 - val_loss: 5.6507 - val_accuracy: 0.0586
Epoch 5/300
192/192 - 6s - loss: 3.7991 - accuracy: 0.1645 - val_loss: 4.8488 - val_accuracy: 0.0790
Epoch 6/300
192/192 - 6s - loss: 3.5210 - accuracy: 0.2112 - val_loss: 5.1321 - val_accuracy: 0.1124
Epoch 7/300
192/192 - 6s - loss: 3.1521 - accuracy: 0.2605 - val_loss: 7.7745 - val_accuracy: 0.0969
Epoch 8/300
192/192 - 6s - loss: 2.9287 - accuracy: 0.3009 - val_loss: 4.4056 - val_accuracy: 0.1319
Epoch 9/300
192/192 - 6s - loss: 2.6687 - accuracy: 0.3480 - val_loss: 4.5217 - val_accuracy: 0.1743
Epoch 10/300
192/192 - 6s - loss: 2.4705 - accuracy: 0.3877 - val_loss: 4.6723 - val_accuracy: 0.1678

Epoch 82/300
192/192 - 6s - loss: 0.0608 - accuracy: 0.9823 - val_loss: 8.5545 - val_accuracy: 0.3469
Epoch 83/300
192/192 - 6s - loss: 0.0763 - accuracy: 0.9795 - val_loss: 11.1489 - val_accuracy: 0.2443
Epoch 84/300
192/192 - 6s - loss: 0.0798 - accuracy: 0.9772 - val_loss: 5.2530 - val_accuracy: 0.3200
Epoch 85/300
192/192 - 6s - loss: 0.0642 - accuracy: 0.9821 - val_loss: 7.1160 - val_accuracy: 0.2980
Epoch 86/300
192/192 - 6s - loss: 0.0668 - accuracy: 0.9805 - val_loss: 8.2080 - val_accuracy: 0.3078
Epoch 87/300
192/192 - 6s - loss: 0.0572 - accuracy: 0.9839 - val_loss: 10.4103 - val_accuracy: 0.2655
Epoch 88/300
192/192 - 6s - loss: 0.0659 - accuracy: 0.9806 - val_loss: 7.2239 - val_accuracy: 0.3404
Epoch 89/300
192/192 - 6s - loss: 0.0807 - accuracy: 0.9792 - val_loss: 4.3012 - val_accuracy: 0.2443
Epoch 90/300
192/192 - 6s - loss: 0.0698 - accuracy: 0.9795 - val_loss: 5.8348 - val_accuracy: 0.3200
Epoch 91/300
192/192 - 6s - loss: 0.0635 - accuracy: 0.9814 - val_loss: 5.4037 -

Epoch 162/300
192/192 - 6s - loss: 0.0262 - accuracy: 0.9922 - val_loss: 5.2782 - val_accuracy: 0.2679
Epoch 163/300
192/192 - 6s - loss: 0.0366 - accuracy: 0.9906 - val_loss: 10.3986 - val_accuracy: 0.3363
Epoch 164/300
192/192 - 6s - loss: 0.0305 - accuracy: 0.9919 - val_loss: 8.2491 - val_accuracy: 0.2606
Epoch 165/300
192/192 - 6s - loss: 0.0419 - accuracy: 0.9886 - val_loss: 9.2940 - val_accuracy: 0.3160
Epoch 166/300
192/192 - 6s - loss: 0.0400 - accuracy: 0.9910 - val_loss: 7.8061 - val_accuracy: 0.3477
Epoch 167/300
192/192 - 6s - loss: 0.0294 - accuracy: 0.9914 - val_loss: 14.3522 - val_accuracy: 0.2777
Epoch 168/300
192/192 - 6s - loss: 0.0303 - accuracy: 0.9923 - val_loss: 8.3398 - val_accuracy: 0.3477
Epoch 169/300
192/192 - 6s - loss: 0.0531 - accuracy: 0.9879 - val_loss: 7.2325 - val_accuracy: 0.2752
Epoch 170/300
192/192 - 6s - loss: 0.0451 - accuracy: 0.9889 - val_loss: 8.5595 - val_accuracy: 0.2956
Epoch 171/300
192/192 - 6s - loss: 0.0411 - accuracy: 0.9904 - val_loss

Epoch 242/300
192/192 - 6s - loss: 0.0242 - accuracy: 0.9943 - val_loss: 6.3589 - val_accuracy: 0.2744
Epoch 243/300
192/192 - 6s - loss: 0.0218 - accuracy: 0.9928 - val_loss: 7.6836 - val_accuracy: 0.2948
Epoch 244/300
192/192 - 6s - loss: 0.0216 - accuracy: 0.9948 - val_loss: 8.1191 - val_accuracy: 0.3298
Epoch 245/300
192/192 - 6s - loss: 0.0367 - accuracy: 0.9932 - val_loss: 8.8334 - val_accuracy: 0.3094
Epoch 246/300
192/192 - 6s - loss: 0.0265 - accuracy: 0.9935 - val_loss: 9.0585 - val_accuracy: 0.2671
Epoch 247/300
192/192 - 6s - loss: 0.0330 - accuracy: 0.9930 - val_loss: 11.5241 - val_accuracy: 0.2875
Epoch 248/300
192/192 - 6s - loss: 0.0358 - accuracy: 0.9923 - val_loss: 7.7730 - val_accuracy: 0.2883
Epoch 249/300
192/192 - 6s - loss: 0.0273 - accuracy: 0.9927 - val_loss: 9.7051 - val_accuracy: 0.3094
Epoch 250/300
192/192 - 6s - loss: 0.0210 - accuracy: 0.9948 - val_loss: 10.3778 - val_accuracy: 0.3559
Epoch 251/300
192/192 - 6s - loss: 0.0312 - accuracy: 0.9919 - val_loss

INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpc9a2lk6f\model\data\model\assets


26/26 - 1s - loss: 7.8195 - accuracy: 0.3366


In [ ]:
!mlflow server --help

In [ ]:
print(mlflow.__version__)

In [ ]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=logs

In [ ]:
def prediction_accuracy(image_test, model):
    #going through each of the test images
    count = 0
    for i in range(len(image_test)): #going through all test images
        img = image_test[i] #load in the image
        
        img = img.reshape(-1, 100, 100, 3) #get it in the right shape for model.predict
        
        prediction = model.predict(img).flatten() #using model predict to get what it thinks is the answer
        
        # prediction.astype(np.int) #converts predictions to integers
        
        pred_index = np.argmax(prediction) # grabs the INDEX of the best prediction
        
        max_pred = max(prediction) # grabs the SCORE of best prediction
        # print(prediction)
        # print(pred_index)
        
        test_label_value = label_test[i][pred_index] # gets the actual test label at the same index
        # print(test_label_value)
        if test_label_value == 1:
            #print("CORRECT PREDICTION")
            count += 1
            
#         # FIND THE INDEX WHERE THE VALUE = 1
#         for j in range(len(predicted_label)):
#             if predicted_label[j] == 1 : #find the point when it finds 1 
#                 if label_test[i][j] == 1: #check if at this exact point is also 1  whic means correct prediction
#                     count+=1 #increase number of correct count by 1
#                     print(count)
#                     break
    print(f"num correct - {count}")
    accuracy = (count/len(label_test))*100
    per_symbol = '%'
    print(f'accuracy is {accuracy} {per_symbol}')        

In [ ]:
image_test.shape

In [ ]:
prediction_accuracy(image_test, model) 

In [ ]:
#checks for best model
runs = mlflow.search_runs(experiment_ids=experiment_id,
                          order_by=['metrics.mae'], max_results=1)

In [ ]:
runs.loc[0]